In [8]:
import time
import warnings

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

warnings.filterwarnings(action='ignore')

In [9]:
option = Options()
option.add_argument('--window-size=1600,900')
driver = webdriver.Chrome('./chromedriver', options=option)
driver.implicitly_wait(5)

driver.get('https://comic.naver.com/comment/comment?titleId=790713&no=95')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reviews = soup.select('span.u_cbox_contents')

# 베댓의 개수 만큼 반복한다.
for i in range(len(reviews[:])):
    print(f'{i + 1:02d}번째 베댓: {reviews[i].text}')

    # 베댓 답글을 연다.
    path = f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{i + 1}]/div[1]/div/div[5]/a'
    driver.find_element_by_xpath(path).click()
    time.sleep(0.5)

    # 베댓 1번째 답글 페이지를 읽는다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    replies = soup.select('div.u_cbox_reply_area span.u_cbox_contents')
    # if len(replies) != 0:
    #     print(f'{i + 1:02d}번째 베댓 답글 개수: {len(replies)}개')

    # 베댓 1번째 답글 페이지에 답글이 있으면 출력한다.
    if len(replies) == 0:
        print('답글 없음')
    else:
        for reply in replies:
            print(f'→ {reply.text}')

    # 베댓 1번째 답글 페이지를 제외한 나머지 답글 페이지 개수를 얻어온다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    replyPage = soup.select('.u_cbox_reply_area .u_cbox_page')
    # print(f'베댓 답글 페이지 개수: {len(replyPage)}개')

    # 1번째 베댓 페이지의 답글은 읽어왔으므로 2번째 베댓 답글 페이지부터 마지막 베댓 답글 페이지까지 반복한다.
    for j in range(1, len(replyPage)):
        # 다음에 읽을 베댓 답글을 연다.
        driver.find_element_by_css_selector(
            f'#cbox_module_wai_u_cbox_content_wrap_tabpanel div.u_cbox_paginate > div > a:nth-child({str(j + 3)})').click()
        time.sleep(0.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        replies = soup.select('div.u_cbox_reply_area span.u_cbox_contents')
        for reply in replies:
            print(f'→ {reply.text}')
    print()

    # 베댓 답글을 닫는다.
    driver.find_element_by_xpath(path).click()
    time.sleep(0.5)